# Quickstart: Using IBM Granite 3.3 on watsonx.ai (with LangChain)

This notebook is a step-by-step tutorial for running **Granite 3.3** models on **IBM watsonx.ai** from your Jupyter environment. It covers prerequisites, how to get your credentials and project set up in IBM Cloud, local installation, environment configuration, choosing the right model, and a minimal Hello World via both the **ibm-watsonx-ai** SDK and **LangChain**.

**What you'll do**
1. Complete IBM Cloud prerequisites (account, API key, watsonx.ai project).
2. Install the required Python packages.
3. Choose a **Granite 3.3** model (e.g., `granite-3-3-8b-instruct`).
4. Make a generation request with the **IBM SDK**.
5. Make the same request via **LangChain**.

> ℹ️ **Granite 3.3** (2B/8B; Base & Instruct) improves reasoning, coding, and instruction following; supports **128K** token context and developer-friendly prompting patterns (e.g., FIM).  
> References: [watsonx.ai foundation model catalog](https://www.ibm.com/products/watsonx-ai/foundation-models), [Granite 3.3 docs](https://www.ibm.com/granite/docs/models/granite/).


## 0) Prerequisites

Before running any code here, complete these steps in IBM Cloud:

1. **Create / sign in to an IBM Cloud account** and provision **watsonx.ai**.  
   - Go to [IBM Cloud](cloud.ibm.com) and sign up or log in with your IBM ID.
   - If you’re new, you can start with a Lite plan or apply a promo code for free credits.
   - Create a Watsonx.ai service instance from your IBM Cloud dashboard. This provides access to IBM foundation models, including Granite 3.3.
   - After login, navigate to the IBM watsonx.ai service from the IBM Cloud Catalog. Go to Catalog → AI & Machine Learning → watsonx.ai.
2. **Create a watsonx.ai Project**
   - In the watsonx.ai console, click Projects → View all projects → New project.
   - Choose:
      - Empty project (start from scratch), or
      - Sample project (with preloaded assets), or
      - Import project (upload a .zip from another environment).
   - Name your project (must be unique, 1–255 chars, no %, \\, or leading/trailing spaces).
   - Storage requirement: Associate an IBM Cloud Object Storage instance (create one if you don’t have it).
   - Click Create. Your project now has:
      - A unique Project ID (needed for API calls).
      - A dedicated storage bucket for assets.
   - Note your service **URL** (region endpoint), e.g., `https://us-south.ml.cloud.ibm.com`. 
3. **Associate Watson Machine Learning Service**
   - Open your project -> Click on Manage at the top nav bar -> Services and Integrations.
   - Click Associate Service -> choose Watson Machine Learning (WML). Note: You must have Admin role in project.
   - If you don’t have a WML instance:
      - Go to IBM Cloud Catalog → Watson Machine Learning.
      - Create an instance in the same region as your project.
      - Once created, link the WML instance to your project.
      - This step is required for running foundation models and APIs.
4. **Obtain Your Credentials**
   - In IBM Cloud, go to Manage → Access (IAM) → [API keys](https://cloud.ibm.com/iam/apikeys).
   - Create an API key (or use an existing one).
   - Download or copy your key immediately (cannot see again after creation).
   - Note:
      - API Key → used for authentication.
      - Service URL → e.g., https://us-south.ml.cloud.ibm.com.
      - Project ID → from your watsonx.ai project details. In watsonx.ai, open left menu > Projects > View all projects. Select your desired project.
      - Go to Manage tab > General. Copy Project ID for later.
      - You should see Watson Machine Learning listed under Services for your project.
      - These three values (API_KEY, URL, PROJECT_ID) are required for SDK and LangChain integration.
      - See: [Credentials for programmatic access](https://www.ibm.com/docs/en/watsonx/saas?topic=resources-credentials-programmatic-access). 
5. Validate  Access
   -  Ensure your IBM Cloud user has Editor role for watsonx.ai and WML services.
   - If you plan to use APIs directly, confirm you can generate a Bearer token using your API key.
6. (Optional) In **Resource hub** or **Prompt Lab**: 
   - review Granite 3.3 models and copy the exact **`model_id`**. You can also list available models via API (shown later).  
   - See: [Get foundation model information / model IDs](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-api-model-ids.html?context=wx&audience=wdp).

> You will paste **three values** into this notebook: `WATSONX_URL`, `WATSONX_APIKEY`, and `PROJECT_ID`.


## 1) Choosing a Granite 3.3 model

**Granite 3.3 variants:**
- **Instruct**: `granite-3-3-2b-instruct`, `granite-3-3-8b-instruct` — best default for apps (chat, Q&A, reasoning, summaries, coding).  
- **Base**: `granite-3-3-2b-base`, `granite-3-3-8b-base` — pre-trained backbone; choose if you plan to fine-tune or do advanced prompting.

**How to choose**
- **2B vs 8B**: 2B is faster/cheaper; **8B** yields stronger reasoning and output quality.  
- **Context**: Granite 3.3 supports **up to 128K tokens**, ideal for long docs and RAG pipelines.  
- **Features**: Better reasoning/math/coding, function/tool calling support patterns, and **FIM** for code completion flows.  
References: [IBM foundation models](https://www.ibm.com/products/watsonx-ai/foundation-models), [Granite 3.3 docs & examples](https://www.ibm.com/granite/docs/models/granite/).

> ⚠️ Availability can be **region-specific**; if a model isn’t found, check your region’s catalog or list models via API (Section 4).


## 2) Install packages locally

Run the next cell to install the SDKs and helpers:


In [ ]:
%pip install ibm-watsonx-ai langchain langchain-ibm python-dotenv

## 3) Configure your local environment

Set the required settings as environment variables for this session. (For reuse across sessions, see the optional `.env` helper in the next cell.)


In [ ]:
import os
from getpass import getpass

# --- Paste your values when prompted ---
WATSONX_URL = os.environ.get("WATSONX_URL") or input("watsonx.ai URL (e.g., https://us-south.ml.cloud.ibm.com): ")
WATSONX_APIKEY = os.environ.get("WATSONX_APIKEY") or getpass("IBM Cloud API Key: ")
PROJECT_ID = os.environ.get("PROJECT_ID") or input("watsonx.ai Project ID: ")

# Persist in this process
os.environ["WATSONX_URL"] = WATSONX_URL
os.environ["WATSONX_APIKEY"] = WATSONX_APIKEY
os.environ["PROJECT_ID"] = PROJECT_ID

print("✅ Environment variables set (not saved to disk).")

### (Optional) Save a local `.env` for reuse
This helper writes your current values into a `.env` file (don’t commit secrets). You can then `source` it or use `python-dotenv`.


In [ ]:
from pathlib import Path, PurePath

env_text = f"""WATSONX_URL={os.environ['WATSONX_URL']}
WATSONX_APIKEY={os.environ['WATSONX_APIKEY']}
PROJECT_ID={os.environ['PROJECT_ID']}
MODEL_ID=granite-3-3-8b-instruct
"""
Path('.env').write_text(env_text)
print('Wrote .env (contains secrets)')

## 4) Hello World with the **ibm-watsonx-ai** SDK

A minimal generation call with a Granite 3.3 Instruct model.  
Docs: [ibm-watsonx-ai Python library](https://www.ibm.com/docs/en/watsonx/saas?topic=resources-python-library)


In [ ]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
import os

MODEL_ID = os.environ.get("MODEL_ID", "ibm/granite-3-3-8b-instruct")  # set to 2B if desired

params = {
    "decoding_method": "sample",
    "max_new_tokens": 128,
    "min_new_tokens": 1,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.9,
}

creds = Credentials(url=os.environ['WATSONX_URL'], api_key=os.environ['WATSONX_APIKEY'])
model = ModelInference(
    model_id=MODEL_ID,
    credentials=creds,
    project_id=os.environ['PROJECT_ID'],
    params=params,
)

prompt = "Say hello to the world in one short sentence."
result = model.generate(prompt)
print(result)

# Pretty-print primary text if present
try:
    print("\n--- Generated text ---\n")
    print(result["results"][0]["generated_text"])
except Exception:
    pass


## 5) Hello World with **LangChain**

LangChain’s `WatsonxLLM` wrapper uses your watsonx **URL**, **project_id**, and **model_id**.  
Docs: [LangChain watsonx integration](https://python.langchain.com/docs/integrations/llms/ibm_watsonx/), [langchain-ibm (PyPI)](https://pypi.org/project/langchain-ibm/)


In [ ]:
from langchain_ibm import WatsonxLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 64,
    "min_new_tokens": 1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 1.0,
}

llm = WatsonxLLM(
    model_id=os.environ.get('MODEL_ID', 'ibm/granite-3-3-8b-instruct'),
    url=os.environ['WATSONX_URL'],
    project_id=os.environ['PROJECT_ID'],
    params=parameters,
)

prompt = PromptTemplate.from_template("""
You are a friendly assistant. Reply very briefly.

Question: {question}
""")

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"question": "Give me a 5-word greeting."}))